In this notebook, we will be plotting the performance of RefCM across all currently available datasets, as well as testing what parameters yield the best performance.

In [1]:
import sys
sys.path.append('../src/')

import os
import json
import config
import logging
import numpy as np
import scanpy as sc
from refcm import RefCM

config.start_logging(logging.DEBUG)

In [2]:
%load_ext autoreload
%autoreload 2

All currently tested datasets, and their associated .obs clustering key:

In [3]:
DSS = {
    # # celltypist datasets
    "Blood": "cell_type",
    "Bone_marrow": "cell_type",
    # "Heart": "cell_type",
    # "Hippocampus": "cell_type",
    # "Intestine": "cell_type",
    # "Kidney": "cell_type",
    # "Liver": "cell_type",
    # "Lung": "cell_type",
    "Lymph_node": "cell_type",
    # "Pancreas": "cell_type",
    "Skeletal_muscle": "cell_type",
    "Spleen": "cell_type",
    
    # # gut atlas
    "gut_atlas_tcell": "annotation",
    
    # Allen-Brain datasets
    "ALM": "labels34",
    "MTG": "labels34",
    "VISp": "labels34",
    
    # pbmc datasets
    "pbmc_10Xv2": "labels",
    "pbmc_10Xv3": "labels",
    "pbmc_CEL-Seq": "labels",
    "pbmc_Drop-Seq": "labels",
    "pbmc_inDrop": "labels",
    "pbmc_Seq-Well": "labels",
    "pbmc_Smart-Seq2": "labels",
    
    # pancreas datasets
    "pancreas_celseq": "celltype",
    "pancreas_celseq2": "celltype",
    "pancreas_fluidigmc1": "celltype",
    "pancreas_indrop1": "celltype",
    "pancreas_indrop2": "celltype",
    "pancreas_indrop3": "celltype",
    "pancreas_indrop4": "celltype",
    "pancreas_smarter": "celltype",
    "pancreas_smartseq2": "celltype",
    
    # LGN datasets
    "LGN_human_intron": "cluster_label",
    "LGN_human_exon": "cluster_label",
    "LGN_macaque_intron": "cluster_label",
    "LGN_macaque_exon": "cluster_label",
    "LGN_mouse_intron": "cluster_label",
    "LGN_mouse_exon": "cluster_label",
}

Use the below codeblock to test out certain parameters (or think how you could automate a grid-like parameter-search) to determine which choice might yield the best results (and why that may be the case?).

In [4]:
# query / reference dataset choice
q_id = 'LGN_human_intron'
ref_id = 'LGN_macaque_intron'

q_ds = sc.read_h5ad(f'../data/{q_id}.h5ad')
ref_ds = sc.read_h5ad(f'../data/{ref_id}.h5ad')

# the parameters to tune
rcm = RefCM(target_sum=1e6, discovery_threshold=0.5, max_splits=3)

# evaluation / display
m = rcm.annotate(q_ds, q_id, ref_ds, ref_id, DSS[q_id], DSS[ref_id])
perf = m.eval(DSS[q_id])
m.display_matching_costs(DSS[q_id])

[h5py._conv      ] [DEBUG   ] : Creating converter from 3 to 5
[refcm           ] [INFO    ] : NOTE: raw counts expected in anndata .X attributes.
[refcm           ] [DEBUG   ] : Loading cached mapping costs from ws_cache.json.
[refcm           ] [DEBUG   ] : Using costs for LGN_human_intron->LGN_macaque_intron found in cache.
[refcm           ] [DEBUG   ] : starting LP optimization
[refcm           ] [DEBUG   ] : optimization terminated w. status "Optimal"
[matchings       ] [DEBUG   ] : [+] GABA1                mapped to GABA1               
[matchings       ] [DEBUG   ] : [+] GABA2                mapped to GABA2               
[matchings       ] [DEBUG   ] : [-] GABA2                mapped to P                   
[matchings       ] [DEBUG   ] : [+] GABA3                mapped to GABA3               
[matchings       ] [DEBUG   ] : [-] GABA3                mapped to GABA4               
[matchings       ] [DEBUG   ] : [-] GABA3                mapped to M                   
[matchings

If you have found better parameters for a certain query -> reference pair, enter these under the `params.json` file. In the example above, this would mean editing line 485 to: 

```json
    ...
    "LGN_human_intron": {
            ...
            "LGN_macaque_intron": {"target_sum": 1000000, "discovery_threshold": 0.5, "max_splits": 3},
            ...
    }
    ...
```

More importantly, think about the characteristics that make one mapping "better" than another. How can you quantify this, and express it as an equation? Once you have found a satisfying definition (equivalently loss or utility function), try to automate searching for the best parameters for all combinations.

In [5]:
# define the loss function associated to 

To see the performance metrics already available to you, check the results of running `m.eval()` (stored in `perf` above). For more documentation, check `src/matchings.py`.

In [6]:
perf

{'common': ['GABA1', 'GABA2', 'GABA3', 'K1', 'K2'],
 'n_common': 5,
 'counts': {'Astro': 49,
  'GABA1': 56,
  'GABA2': 60,
  'GABA3': 26,
  'K1': 65,
  'K2': 39,
  'MP': 706,
  'Micro': 22,
  'OPC': 24,
  'Oligo': 162},
 'counts_total': 1209,
 'counts_total_common': 246,
 'counts_total_noncommon': 963,
 'n_correct_links': 4,
 'n_incorrect_links': 5,
 'n_notmapped_links': 81,
 'common_w_correct_links': ['GABA1', 'GABA2', 'GABA3', 'K2'],
 'n_common_w_correct_links': 4,
 'pct_common_w_correct_links': 0.7357723577235772,
 'common_w_only_correct_links': ['GABA1'],
 'n_common_w_only_correct_links': 1,
 'pct_common_w_only_correct_links': 0.22764227642276422,
 'common_w_incorrect_links': ['GABA2', 'GABA3', 'K1', 'K2'],
 'n_common_w_incorrect_links': 4,
 'pct_common_w_incorrect_links': 0.7723577235772358,
 'common_w_only_incorrect_links': ['K1'],
 'n_common_w_only_incorrect_links': 1,
 'pct_common_w_only_incorrect_links': 0.26422764227642276,
 'common_w_correct_and_incorrect_links': ['GABA2', '

With the parameters available in `params.json`, we can then view our performance across all dataset combinations as follows:

In [7]:
with open('params.json', 'r') as f:
    params = json.load(f)

In [8]:
# performance log across the datasets
perfs = [[None for _ in DSS] for _ in DSS]


# mapping all different combinations
for i, (a_id, a_key) in enumerate(DSS.items()):
    a_ds = sc.read_h5ad(f'../data/{a_id}.h5ad')
    
    for j, (b_id, b_key) in enumerate(DSS.items()):
        if j < i:
            continue
        
        b_ds = sc.read_h5ad(f'../data/{b_id}.h5ad')
        
        # run first with a as query and b as reference, then b as query and a as reference
        # to avoid re-reading certain files as often.
        
        rcm = RefCM(**params[a_id][b_id])
        m = rcm.annotate(a_ds, a_id, b_ds, b_id, DSS[b_id], DSS[a_id])
        perfs[i][j] = m.eval(ground_truth_obs_key=DSS[a_id])
        
        if i == j:
            continue
        
        rcm = RefCM(**params[b_id][a_id])
        m = rcm.annotate(b_ds, b_id, a_ds, a_id, DSS[a_id], DSS[b_id])
        perfs[j][i] = m.eval(ground_truth_obs_key=DSS[b_id])

[refcm           ] [INFO    ] : NOTE: raw counts expected in anndata .X attributes.
[refcm           ] [DEBUG   ] : Loading cached mapping costs from ws_cache.json.
[refcm           ] [DEBUG   ] : Selecting joint gene subset for query and reference datasets
[refcm           ] [DEBUG   ] : Using 1654 genes.
[refcm           ] [DEBUG   ] : Computing Wasserstein distances.
|████████████████| [100.00% ] : 38:33
[refcm           ] [DEBUG   ] : Saving Blood->Blood mapping costs to ws_cache.json.
[refcm           ] [DEBUG   ] : starting LP optimization
[refcm           ] [DEBUG   ] : optimization terminated w. status "Optimal"
[matchings       ] [DEBUG   ] : [+] CD16-negative, CD56-bright natural killer cell, human mapped to CD16-negative, CD56-bright natural killer cell, human
[matchings       ] [DEBUG   ] : [+] CD16-positive, CD56-dim natural killer cell, human mapped to CD16-positive, CD56-dim natural killer cell, human
[matchings       ] [DEBUG   ] : [+] central memory CD4-positive, alpha

In [9]:
from typing import Dict, Tuple, List, Callable

metric_name = "pct_common_clusters_correctly_linked"
def metric(perf_log: Dict) -> float:
    return perf_log['n_common_w_correct_links'] / perf_log['n_common'] if perf_log['n_common'] != 0 else 1.0

In [10]:
import plotly.express as px

def display_performance(perfs: List[List[Dict]], metric_name: str, metric: Callable[[Dict], float]) -> None:
    
        title = metric_name

        y_label = "Query"
        x_label = "Reference"
        
        ps = list(map(lambda row: list(map(metric, row)), perfs))

        # plot the matching costs
        fig = px.imshow(
            ps,
            title=title,
            labels=dict(y=y_label, x=x_label, color="performance"),
            x=list(DSS.keys()),
            y=list(DSS.keys()),
            color_continuous_scale="Agsunset",
        )
        fig.update_xaxes(tickangle=-45)

        fig.show()

In [15]:
display_performance(perfs, metric_name, metric)

In [17]:
for i in range(len(DSS)):
    for j in range(len(DSS)):
        if metric(perfs[i][j]) != metric(perfs[j][i]):
            print(i, list(DSS.keys())[i], j, list(DSS.keys())[j])

0 Blood 2 Lymph_node
1 Bone_marrow 2 Lymph_node
2 Lymph_node 0 Blood
2 Lymph_node 1 Bone_marrow
